# **AML ASSIGNMENT 1**
**Name: Nilanjan Debnath**

**Roll No.: MDS201919**

# **Importing Packages**

Here we import pandas, numpy and various layers from keras which will help us in defining the neural network

In [ ]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from tensorflow import keras
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D, Dense , Flatten, Dropout , BatchNormalization
from keras.optimizers import Adam

# **Downloading the dataset**

We have already uploaded the dataset into google drive. In this section we download the dataset for us to use.

In [ ]:
X = np.load("drive/MyDrive/X.npy")

In [ ]:
Y = np.load("drive/MyDrive/Y.npy")

We make the test dataset 20% of the training dataset

In [ ]:
x_train, x_test, y_train, y_test = train_test_split(X,Y,test_size=0.2,random_state=42)
x_train = x_train.reshape(-1,64,64,1)
x_test = x_test.reshape(-1,64,64,1)

# **Importing the Bayesian Optimization program**

In [ ]:
from google.colab import files
src = list(files.upload().values())[0]
open('gp.py','wb').write(src)
import gp

%load gp.py
%run gp.py

Saving gp.py to gp.py


# **Creating the base model**

Here we create the base of the model we want to tune the hyperparameters of. The parameters we are tuning are the number of convolutional layers, number of filters in each convolutional layer, number of max pooling layers, number of BatchNormalization layers, number of dropout layers, number of dense layers and number of nodes in each dense layers

In [ ]:
def create_model(num_layers,num_filters,num_dense_layers,num_dense_nodes):
  model = Sequential()
  model.add(Conv2D(filters=32,kernel_size=(3,3),activation="relu",padding="same",input_shape=(64,64,1)))
  model.add(MaxPooling2D((2, 2)))
  model.add(BatchNormalization())
  model.add(Dropout(0.25))
  for i in range(num_layers):
        model.add(Conv2D(filters=num_filters,kernel_size=(3,3),activation='relu',padding="same"))
        model.add(MaxPooling2D(pool_size=(2,2),padding="same"))
        model.add(BatchNormalization())
        model.add(Dropout(0.25))
  model.add(Flatten())
  for i in range(num_dense_layers):
        model.add(Dense(num_dense_nodes,activation = 'relu'))
  model.add(Dense(10,activation= 'softmax'))
  adam = Adam(lr=1e-3, decay= 1e-3)
  model.compile(optimizer=adam,loss='categorical_crossentropy',metrics=["accuracy"])
  return model

Here we take a random model with 1 CNN layer, 1 Dropout layer, 1 BatchNormalization layer, 1 MaxPooling layer, 1 Dense layer with 128 nodes. Then we find its accuracy on the test data

In [ ]:
AMLclassifier1 = create_model(1,32,1,64)

We fit the model with 40 epochs and split the training set to make a validation set which contains 15% of the original training set

In [ ]:
AMLclassifier1.fit(x=x_train,
                   y=y_train,
                   epochs=40,
                   validation_split = 0.15)
                              

Epoch 1/40
44/44 [==============================] - 0s 11ms/step - loss: 1.8056 - accuracy: 0.4154 - val_loss: 2.2655 - val_accuracy: 0.0887
Epoch 2/40
44/44 [==============================] - 0s 6ms/step - loss: 0.7279 - accuracy: 0.7616 - val_loss: 2.5691 - val_accuracy: 0.2661
Epoch 3/40
44/44 [==============================] - 0s 6ms/step - loss: 0.4244 - accuracy: 0.8565 - val_loss: 2.8242 - val_accuracy: 0.2056
Epoch 4/40
44/44 [==============================] - 0s 6ms/step - loss: 0.3105 - accuracy: 0.8979 - val_loss: 3.0569 - val_accuracy: 0.2903
Epoch 5/40
44/44 [==============================] - 0s 6ms/step - loss: 0.2235 - accuracy: 0.9300 - val_loss: 3.5958 - val_accuracy: 0.1694
Epoch 6/40
44/44 [==============================] - 0s 6ms/step - loss: 0.1654 - accuracy: 0.9557 - val_loss: 3.6308 - val_accuracy: 0.1734
Epoch 7/40
44/44 [==============================] - 0s 6ms/step - loss: 0.1076 - accuracy: 0.9679 - val_loss: 3.4661 - val_accuracy: 0.2540
Epoch 8/40
44/44 [=

We evaluate our model on the test set

In [ ]:
test_scores=AMLclassifier1.evaluate(x_test, y_test, verbose=0)

In [ ]:
print("Test dataset accuracy:{0} ",test_scores[1]*100,"%")

Test dataset accuracy:{0}  89.58837985992432 %


As we can see that our accuracy is 89.59%. Let's try to see if we can make it better than that using Bayesian Optimization

# **Using Bayesian Optimization**

At first we define the function Bayes Optimization is supposed to optimize. We decide on optimizing accuracy

In [ ]:
def fitness(params):

    model = create_model(num_layers = np.int(params[0]),
                         num_filters = np.int(params[1]),
                         num_dense_layers=np.int(params[2]),
                         num_dense_nodes=np.int(params[3])
                        )
    

    #named blackbox becuase it represents the structure
    blackbox = model.fit(x=x_train,
                        y=y_train,
                        epochs=40,
                        validation_split = 0.15)
                        
    #return the validation accuracy for the last epoch.
    accuracy = blackbox.history['val_accuracy'][-1]

    # Print the classification accuracy.
    print()
    print("Accuracy: {0:.2%}".format(accuracy))
    print()
    #print the parameters
    print(params)
    del model
    
    return accuracy

Here we set the bounds to the number of nodes and layers we want

In [ ]:
bounds = np.array([[1, 10],[16,128], [0,5], [64,256]])

Now we find the best hyperparameters

In [ ]:
xp, yp = bayesian_optimisation(n_iters=25, sample_loss=fitness, 
                               bounds=bounds,
                               n_pre_samples=10)

Epoch 1/40
44/44 [==============================] - 1s 15ms/step - loss: 2.2992 - accuracy: 0.2684 - val_loss: 2.3370 - val_accuracy: 0.1169
Epoch 2/40
44/44 [==============================] - 0s 8ms/step - loss: 1.4089 - accuracy: 0.5096 - val_loss: 2.7175 - val_accuracy: 0.1169
Epoch 3/40
44/44 [==============================] - 0s 7ms/step - loss: 0.9409 - accuracy: 0.6824 - val_loss: 3.4335 - val_accuracy: 0.1169
Epoch 4/40
44/44 [==============================] - 0s 7ms/step - loss: 0.7293 - accuracy: 0.7480 - val_loss: 3.8009 - val_accuracy: 0.1169
Epoch 5/40
44/44 [==============================] - 0s 7ms/step - loss: 0.5464 - accuracy: 0.8201 - val_loss: 3.8990 - val_accuracy: 0.1169
Epoch 6/40
44/44 [==============================] - 0s 7ms/step - loss: 0.4507 - accuracy: 0.8580 - val_loss: 4.0939 - val_accuracy: 0.1169
Epoch 7/40
44/44 [==============================] - 0s 7ms/step - loss: 0.3910 - accuracy: 0.8744 - val_loss: 3.6339 - val_accuracy: 0.1169
Epoch 8/40
44/44 [=

We see the best hyperparameters turn out to be 9 layers of Convolutional layers, 89 filters for each Convolutional layer, 9 layers of maxpooling, 9 batch normalization layers, 9 dropout layers, 5 dense layers with 196 nodes

In [ ]:
xp_hat = np.round(xp[np.array(yp).argmax(), :])
print(np.round(xp_hat))

[  9.  89.   5. 196.]


# **Testing the best model as estimated by Bayesian Hyperparameter tuning algorithm**

Now we create the model with said hyperparameters

In [ ]:
best = create_model(9,89,5,196)

After creation of the model, we now train the weights. We set a validation split of 15% similar to what we defined in the earlier model.

In [ ]:
best.fit(x=x_train,
         y=y_train,
         epochs=40,
         validation_split = 0.15)

Epoch 1/40
44/44 [==============================] - 1s 22ms/step - loss: 2.3162 - accuracy: 0.0921 - val_loss: 2.3023 - val_accuracy: 0.1008
Epoch 2/40
44/44 [==============================] - 0s 11ms/step - loss: 2.2069 - accuracy: 0.1570 - val_loss: 2.3067 - val_accuracy: 0.1008
Epoch 3/40
44/44 [==============================] - 0s 11ms/step - loss: 1.8488 - accuracy: 0.2955 - val_loss: 2.3533 - val_accuracy: 0.0968
Epoch 4/40
44/44 [==============================] - 0s 11ms/step - loss: 1.5930 - accuracy: 0.3869 - val_loss: 2.5361 - val_accuracy: 0.0968
Epoch 5/40
44/44 [==============================] - 0s 11ms/step - loss: 1.3046 - accuracy: 0.5139 - val_loss: 2.6137 - val_accuracy: 0.1250
Epoch 6/40
44/44 [==============================] - 0s 11ms/step - loss: 1.1665 - accuracy: 0.5603 - val_loss: 2.7468 - val_accuracy: 0.1169
Epoch 7/40
44/44 [==============================] - 0s 11ms/step - loss: 0.9164 - accuracy: 0.6667 - val_loss: 3.8556 - val_accuracy: 0.1008
Epoch 8/40
44

After training the weights, we now evaluate the accuracy the model gives on the test dataset

In [ ]:
test_scores=best.evaluate(x_test, y_test, verbose=0)

In [ ]:
print("Accuracy on test dataset: ",test_scores[1]*100,"%")

Accuracy on test dataset:  98.0629563331604 %


**Final accuracy after hyperparameter tuning: 98.06% which is a considerable improvement on the 89.59% we were getting without tuning the model**